# Analysing Oxford Area to invest on property market

## Background

The housing market is under the pressure at the moment with number of sales going down. According to Knight Frank the market will see about 3% drop during pandemic, but then it will go up by 5% and further. 
With mortages rates low it is a opportunity to invest in the property market. 
The valuation of real estate comes from traditional and non-traditional feautures of the property. Traditional are being size, type of the property, year build etc. Non-traditional are proximity to points of interests, number cafe and reataraunts, hotels, transportation etc.
Accroding to McKinsey reserch up to 60% of property performance can come from non-traditional feautures.

## Business Problem

Current lockdown restriction on people's moves makes impossible to evaluate the housing area. Even during noraml period it will take a long time to understand the area for potential investment. Data Science methodology can help to understand the area and accelerate the research for the investment decision making.

## Data Section

We will create a data frame of local housing prices to Luckily the housing market data is publicly available from the government data base https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads. We can download the data containing 
- Postcode 
- PAON (Primary Addressable Object Name)
- Typically the house number or name
- SAON (Secondary Addressable Object Name.If there is a sub-building, for example, the building is divided into flats, there will be a SAON.)
- Street
- Locality
- Town/City
- District
- County

To discover local amenities and facilities we will use FourSquare API. We will created a data frame of local venues and visualize it. We will then combine FourSquare API and housing data frames and cluster them to define the best areas for buying a property.

## Workflow

The following workflow will be used in this project.

    1. Data collection
    2. Data inspection and understanding
    3. Data preparation and processing
    4. Data modelling using k-Nearest Neighbor Algorithm
    5. Visualization
    6. Summary of the project results



## 1. Data collection

Let's install all necessary libraries and download the data set. I will be using the data set for 2019 as it is should have better representation of prices and enough number of transactions to explore.


In [6]:
#Importing all neccessary libraries
import numpy as np
import pandas as pd
import datetime as dt
import requests

#to handle JSON files
import json
from pandas.io.json import json_normalize

#To handle location coordinates
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

#import Matplotlib for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

#for maps intsall Folium
!conda install -c conda-forge folium=0.5.0 --yes

print('All libraries installed and imported')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
All libraries installed and imported


In [0]:
#Let's import housing sales data from Land registry data base and create data frame
df_land = pd.read_csv("http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv")

## 2. Data inspection and understanding

Let's have a look at the data and understand it first.


In [8]:
#display first 5 rows of the data set
df_land.head()


,{8F1B26BD-60CA-53DB-E053-6C04A8C03649},221950,2019-04-26 00:00,TS17 5FF,D,Y,F,3,Unnamed: 8,CARRAWBURGH ROAD,INGLEBY BARWICK,STOCKTON-ON-TEES,STOCKTON-ON-TEES.1,STOCKTON-ON-TEES.2,A,A.1
0,{8F1B26BD-60CB-53DB-E053-6C04A8C03649},246995,2019-03-29 00:00,TS15 9ZH,D,Y,F,16,NaN,GRESLEY CLOSE,NaN,YARM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
1,{8F1B26BD-60CC-53DB-E053-6C04A8C03649},244950,2019-05-17 00:00,TS18 2FN,T,Y,F,13,NaN,INFINITY VIEW,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
2,{8F1B26BD-60CD-53DB-E053-6C04A8C03649},139950,2019-05-31 00:00,TS18 2FN,S,Y,F,40,NaN,INFINITY VIEW,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
3,{8F1B26BD-60CE-53DB-E053-6C04A8C03649},271995,2019-05-31 00:00,TS15 9FD,D,Y,F,27,NaN,MALLARD DRIVE,NaN,YARM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
4,{8F1B26BD-60CF-53DB-E053-6C04A8C03649},84450,2019-04-26 00:00,TS18 2FD,T,Y,F,117,NaN,DEEPDALE AVENUE,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A


In [9]:
#find out how many rows and columns data set contains
df_land.shape

(963682, 16)


## 3. Data preparation and processing

The data set from Lang registry contains 16 columns and 963682 rows We will need to re-name the columns to a sensible names, filter the location to Oxford, format date column, group data by post code and calculate average price, assign geographical coordinates to each location and plot the data.


In [10]:
#first we need meaningful names to the headers of the columns
df_land.columns = ['Transaction', 'Price', 'Date', 'Postcode', 'Type', 'Newbuild', 'Duration', 'PAON', 'SAON', 'Street', 
                   'Neighborhood', 'Town', 'District', 'County', 'Cat_Type', 'Record']

df_land.head()

,Transaction,Price,Date,Postcode,Type,Newbuild,Duration,PAON,SAON,Street,Neighborhood,Town,District,County,Cat_Type,Record
0,{8F1B26BD-60CB-53DB-E053-6C04A8C03649},246995,2019-03-29 00:00,TS15 9ZH,D,Y,F,16,NaN,GRESLEY CLOSE,NaN,YARM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
1,{8F1B26BD-60CC-53DB-E053-6C04A8C03649},244950,2019-05-17 00:00,TS18 2FN,T,Y,F,13,NaN,INFINITY VIEW,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
2,{8F1B26BD-60CD-53DB-E053-6C04A8C03649},139950,2019-05-31 00:00,TS18 2FN,S,Y,F,40,NaN,INFINITY VIEW,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
3,{8F1B26BD-60CE-53DB-E053-6C04A8C03649},271995,2019-05-31 00:00,TS15 9FD,D,Y,F,27,NaN,MALLARD DRIVE,NaN,YARM,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A
4,{8F1B26BD-60CF-53DB-E053-6C04A8C03649},84450,2019-04-26 00:00,TS18 2FD,T,Y,F,117,NaN,DEEPDALE AVENUE,NaN,STOCKTON-ON-TEES,STOCKTON-ON-TEES,STOCKTON-ON-TEES,A,A


In [11]:
#next step is to filter data set to Oxford only and see the size of the data frame
df_oxf = df_land.query("Town == 'OXFORD'")
df_oxf.shape

(1831, 16)

In [12]:
df_oxf

,Transaction,Price,Date,Postcode,Type,Newbuild,Duration,PAON,SAON,Street,Neighborhood,Town,District,County,Cat_Type,Record
3381,{919FEC06-2C95-9A90-E053-6C04A8C0A300},145000,2019-07-24 00:00,OX1 2HS,O,N,L,"DARTINGTON HOUSE, 55",UNIT 3,LITTLE CLARENDON STREET,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A
3382,{919FEC06-2C99-9A90-E053-6C04A8C0A300},110000,2019-06-07 00:00,OX3 9ED,O,N,L,146,NaN,LONDON ROAD,HEADINGTON,OXFORD,OXFORD,OXFORDSHIRE,B,A
5054,{919FEC06-2CBE-9A90-E053-6C04A8C0A300},309000,2019-08-02 00:00,OX4 3QH,T,N,F,44,NaN,BARTHOLOMEW ROAD,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A
5055,{919FEC06-2CBF-9A90-E053-6C04A8C0A300},275000,2019-08-05 00:00,OX4 4QJ,S,N,F,12,NaN,BROCKLESBY ROAD,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A
5057,{919FEC06-2CC1-9A90-E053-6C04A8C0A300},400000,2019-08-16 00:00,OX2 6UH,F,N,L,9,FLAT A,LINTON ROAD,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962602,{919FEC06-2C7C-9A90-E053-6C04A8C0A300},361000,2019-08-09 00:00,OX1 1HQ,F,N,L,BOOKBINDERS COURT,APARTMENT 3,ST THOMAS STREET,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A
962603,{919FEC06-2C7D-9A90-E053-6C04A8C0A300},2400000,2019-07-23 00:00,OX2 6UP,S,N,F,16,NaN,NORTHMOOR ROAD,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A
962604,{919FEC06-2C7F-9A90-E053-6C04A8C0A300},975000,2019-08-13 00:00,OX2 7HR,O,N,F,249,NaN,BANBURY ROAD,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A
962606,{919FEC06-2C81-9A90-E053-6C04A8C0A300},910000,2019-07-16 00:00,OX2 6AQ,F,N,L,FOUNDRY HOUSE EAGLE WORKS,72,WALTON WELL ROAD,NaN,OXFORD,OXFORD,OXFORDSHIRE,B,A


In [13]:
#our data set has non defined neighborhood which belongs to Oxford itself
#let's replace NaN valuse with the city name
df_oxf['Neighborhood'] = df_oxf['Neighborhood'].fillna('OXFORD')
df_oxf

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Transaction,Price,Date,Postcode,Type,Newbuild,Duration,PAON,SAON,Street,Neighborhood,Town,District,County,Cat_Type,Record
3381,{919FEC06-2C95-9A90-E053-6C04A8C0A300},145000,2019-07-24 00:00,OX1 2HS,O,N,L,"DARTINGTON HOUSE, 55",UNIT 3,LITTLE CLARENDON STREET,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A
3382,{919FEC06-2C99-9A90-E053-6C04A8C0A300},110000,2019-06-07 00:00,OX3 9ED,O,N,L,146,NaN,LONDON ROAD,HEADINGTON,OXFORD,OXFORD,OXFORDSHIRE,B,A
5054,{919FEC06-2CBE-9A90-E053-6C04A8C0A300},309000,2019-08-02 00:00,OX4 3QH,T,N,F,44,NaN,BARTHOLOMEW ROAD,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A
5055,{919FEC06-2CBF-9A90-E053-6C04A8C0A300},275000,2019-08-05 00:00,OX4 4QJ,S,N,F,12,NaN,BROCKLESBY ROAD,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A
5057,{919FEC06-2CC1-9A90-E053-6C04A8C0A300},400000,2019-08-16 00:00,OX2 6UH,F,N,L,9,FLAT A,LINTON ROAD,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962602,{919FEC06-2C7C-9A90-E053-6C04A8C0A300},361000,2019-08-09 00:00,OX1 1HQ,F,N,L,BOOKBINDERS COURT,APARTMENT 3,ST THOMAS STREET,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A
962603,{919FEC06-2C7D-9A90-E053-6C04A8C0A300},2400000,2019-07-23 00:00,OX2 6UP,S,N,F,16,NaN,NORTHMOOR ROAD,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A
962604,{919FEC06-2C7F-9A90-E053-6C04A8C0A300},975000,2019-08-13 00:00,OX2 7HR,O,N,F,249,NaN,BANBURY ROAD,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A
962606,{919FEC06-2C81-9A90-E053-6C04A8C0A300},910000,2019-07-16 00:00,OX2 6AQ,F,N,L,FOUNDRY HOUSE EAGLE WORKS,72,WALTON WELL ROAD,OXFORD,OXFORD,OXFORD,OXFORDSHIRE,B,A


In [14]:
#grouping the results by neighborhood and average price
df_ox_price = df_oxf.groupby(['Street', 'Postcode'])['Price'].mean().reset_index()
df_ox_price.columns = ['Street', 'Postcode','Price_av']
df_ox_price

,Street,Postcode,Price_av
0,ABBEY ROAD,OX2 0AD,640000.0
1,ABINGDON ROAD,OX1 4TJ,360000.0
2,ABINGDON ROAD,OX2 9QN,455000.0
3,ABLETT CLOSE,OX4 1XH,355000.0
4,ACRE CLOSE,OX3 7FQ,368000.0
...,...,...,...
1173,YEW CLOSE,OX4 7UX,160000.0
1174,YORK AVENUE,OX3 8NS,325000.0
1175,YORK PLACE,OX4 1YL,581000.0
1176,YORK ROAD,OX3 8NP,600000.0


In [15]:
#Download Location coordinates for Oxford
address = 'Oxford, UK'
geolocator = Nominatim(user_agent = "oxy")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geografical location of Oxford is {}, {}.'.format(latitude, longitude))

The geografical location of Oxford is 51.7520131, -1.2578499.


In [16]:
#find out geografical location of postcodes
#download UK post codes geografical locations from open source
!wget https://www.freemaptools.com/download/full-postcodes/ukpostcodes.zip

--2020-05-10 08:02:01--  https://www.freemaptools.com/download/full-postcodes/ukpostcodes.zip
Resolving www.freemaptools.com (www.freemaptools.com)... 208.97.137.162, 2607:f298:5:101a::e06:9b6c
Connecting to www.freemaptools.com (www.freemaptools.com)|208.97.137.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33130126 (32M) [application/zip]
Saving to: ‘ukpostcodes.zip’

ukpostcodes.zip     100%[===================>]  31.59M  9.39MB/s    in 4.6s    

2020-05-10 08:02:07 (6.86 MB/s) - ‘ukpostcodes.zip’ saved [33130126/33130126]



In [17]:
#Create data frame with locations coordinates
loc = pd.read_csv('ukpostcodes.zip')
loc.columns = ['id', 'postcode', 'latitude', 'longitude']
loc.head()

,id,postcode,latitude,longitude
0,1,AB10 1XG,57.144165,-2.114848
1,2,AB10 6RN,57.137880,-2.121487
2,3,AB10 7JB,57.124274,-2.127190
3,4,AB11 5QN,57.142701,-2.093295
4,5,AB11 6UL,57.137547,-2.112233


In [18]:
#let's remove id column
loc = loc.drop(columns = ['id'])
loc.head()

,postcode,latitude,longitude
0,AB10 1XG,57.144165,-2.114848
1,AB10 6RN,57.137880,-2.121487
2,AB10 7JB,57.124274,-2.127190
3,AB11 5QN,57.142701,-2.093295
4,AB11 6UL,57.137547,-2.112233


In [19]:
#and standardize the header
col = ['Postcode', 'Latitude', 'Longitude']
loc.columns = col
loc.head()

,Postcode,Latitude,Longitude
0,AB10 1XG,57.144165,-2.114848
1,AB10 6RN,57.137880,-2.121487
2,AB10 7JB,57.124274,-2.127190
3,AB11 5QN,57.142701,-2.093295
4,AB11 6UL,57.137547,-2.112233


In [20]:
#now let' merge two data frames to match the post code
oxford_data = pd.merge(df_ox_price,
                 loc[['Postcode', 'Latitude', 'Longitude']],
                 on='Postcode')
oxford_data.head()

,Street,Postcode,Price_av,Latitude,Longitude
0,ABBEY ROAD,OX2 0AD,640000.0,51.754412,-1.271848
1,ABINGDON ROAD,OX1 4TJ,360000.0,51.734051,-1.249716
2,ABINGDON ROAD,OX2 9QN,455000.0,51.733564,-1.331408
3,ABLETT CLOSE,OX4 1XH,355000.0,51.749103,-1.233127
4,ACRE CLOSE,OX3 7FQ,368000.0,51.744058,-1.197857


Thus we have got the data frame with postcodes matching to locations coordinates.

Now to narrow our research let's chose properties with a price close to avearge range. The average price for Oxford £500k accourding to real estate agents portal Zoopla.

In [23]:
#Average price for property in Oxford is £500k. Let's chose properties between £450 and £550k
ox_av = oxford_data.query("(Price_av >= 450000) & (Price_av <= 550000)")
ox_av

,Street,Postcode,Price_av,Latitude,Longitude
2,ABINGDON ROAD,OX2 9QN,455000.0,51.733564,-1.331408
5,ACREMEAD ROAD,OX33 1NZ,487500.0,51.745725,-1.154707
10,ALDRICH ROAD,OX2 7SS,492500.0,51.785151,-1.261481
28,ARLINGTON DRIVE,OX3 0SL,497500.0,51.768291,-1.242917
33,ASHCROFT CLOSE,OX2 9SE,492500.0,51.750896,-1.312104
...,...,...,...,...,...
1133,WOLSEY ROAD,OX2 7TA,468000.0,51.787217,-1.264100
1135,WOOD AVENS WAY,OX3 9GA,545000.0,51.771410,-1.216918
1145,WOODPERRY ROAD,OX3 9UZ,550000.0,51.793918,-1.176281
1147,WOODSTOCK CLOSE,OX2 8DB,457500.0,51.784841,-1.273750


In [24]:
#next step let create map of Oxford and vicinity with average property prices per post code as markers
import folium
map_ox = folium.Map(location = [latitude, longitude], zoom_start = 13)

#add markers to the map
for lat, lng, street, postcode in zip(ox_av['Latitude'], ox_av['Longitude'], 
                                          ox_av['Postcode'], ox_av['Street']):
    label = '{}, {}'.format(postcode, street)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng], radius = 5, popup = label, color = 'blue', 
                        fill = True, fill_color = '#3186cc', fill_opacity = 0.7, parse_html = False).add_to(map_ox)
map_ox

Looking at the spread of properties around the city we can see that the properties in the price range we have chosen are evenly distributed. Now we can focus our study on the venues available around the city to chose the area with the most diverse venues categories.

### Using FourSqure API to create venues data frame

We will downolad the venues available arout Oxford city from FourSquare API and will create the data fame of the most common venues. 

In [28]:
#define FourSquare Credentials
CLIENT_ID = '444DKD1SWHVB3YEZGMRHVDINOLU5MMHBHIJ0FWZ5WLRVKUD2'
CLIENT_SECRET = 'BUCGHUPKJEEE4XAKFSYC2UNDZ3R5F5IAQYMBTCEOCFDGBAXN'
VERSION = '20200501'
print('CLIENT ID:' +CLIENT_ID)

CLIENT ID:444DKD1SWHVB3YEZGMRHVDINOLU5MMHBHIJ0FWZ5WLRVKUD2


In [29]:
#Let's see the first street in the data frame
ox_av.loc[2, 'Street']

'ABINGDON ROAD'

In [30]:
street_name = ox_av.loc[2, 'Street']
street_latitude = ox_av.loc[2, 'Latitude']
street_longitude = ox_av.loc[2, 'Longitude']

print('Coordinates of street {} are {}, {}.'.format(street_name, street_latitude, street_longitude))

Coordinates of street ABINGDON ROAD are 51.733564065696, -1.33140848984531.


In [31]:
#get top 100 venues in Oxford
LIMIT = 100
radius = 2000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit{}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    street_latitude,
    street_longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=444DKD1SWHVB3YEZGMRHVDINOLU5MMHBHIJ0FWZ5WLRVKUD2&client_secret=BUCGHUPKJEEE4XAKFSYC2UNDZ3R5F5IAQYMBTCEOCFDGBAXN&v=20200501&ll=51.733564065696,-1.33140848984531&radius=2000&limit100'

In [32]:
#send GET request and examaine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb7b75a618f43001b8014ed'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d433e1d15aa8cfab814fb99-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/gastropub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d155941735',
         'name': 'Gastropub',
         'pluralName': 'Gastropubs',
         'primary': True,
         'shortName': 'Gastropub'}],
       'id': '4d433e1d15aa8cfab814fb99',
       'location': {'address': '28 Appleton Rd.',
        'cc': 'GB',
        'city': 'Cumnor',
        'country': 'United Kingdom',
        'distance': 369,
        'formattedAddress': ['28 Appleton Rd.',
         'Cumnor',
         'Oxfordshire',
         'OX2 9QH',
         'United Kingdom'],
        'labeledLatLngs': [{'label': 

In [0]:
#define function that extracs the category of the venue
def get_category_type(row):
  try:
    categories_list = row['categories']
  except:
    categories_list = row['venue.categories']

  if len(categories_list) == 0:
    return None
  else:
    return categories_list[0]['name']

In [34]:
#create pandas data frame from json file
 venues = results['response']['groups'][0]['items']
 nearby_venues = json_normalize(venues)

 filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
 nearby_venues = nearby_venues.loc[:, filtered_columns]

 nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)
 nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
 nearby_venues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,The Bear & Ragged Staff,Gastropub,51.733540,-1.336767
1,The Vine Inn,Pub,51.733613,-1.331282
2,Link Security,Home Service,51.730737,-1.331938
3,Hid's Copse,Forest,51.746379,-1.314662
4,Denton Green,Park,51.746820,-1.315124
5,Long Copse,Forest,51.739290,-1.316876
6,Blind Pinnock Ruins,Historic Site,51.749195,-1.324825


In [35]:
#now we will see how many venues were returned by FourSquare
print('{} venues were returned by FourSquare'.format(nearby_venues.shape[0]))

7 venues were returned by FourSquare


After extracting venues for one of teh locations let's do the same process for all of the city. I'm using a large radius as there is no enough venues in a smaller radius

In [0]:
#the function retrieves venues given the names and coordinates and stores it into data frame
def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    ox_venues = pd.DataFrame([item for venues_list in venues_list for item in venues_list])
    ox_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(ox_venues)

In [38]:
ox_venues = getNearbyVenues(names=ox_av['Street'],
                              latitudes = ox_av['Latitude'],
                              longitudes = ox_av['Longitude'])

ABINGDON ROAD
ACREMEAD ROAD
ALDRICH ROAD
ARLINGTON DRIVE
ASHCROFT CLOSE
BANBURY ROAD
BERTIE ROAD
BISHOP KIRK PLACE
BLUEBELL MEWS
BOTLEY ROAD
BOTLEY ROAD
BOWNESS AVENUE
BROADHURST GARDENS
BULLINGDON ROAD
BUSHY CLOSE
CHURCH ROAD
CHURCH ROAD
COLTERNE CLOSE
CONISTON AVENUE
COPPOCK CLOSE
CORNWALLIS ROAD
COTSWOLD CRESCENT
COURTLAND ROAD
COWLEY ROAD
COX LANE
CROWN ROAD
CUMNOR HILL
CUMNOR HILL
CUMNOR ROAD
CUNLIFFE CLOSE
DERWENT AVENUE
DIVINITY ROAD
DUKE STREET
EAST AVENUE
EAST FIELD CLOSE
EDGEWAY ROAD
EGERTON ROAD
EWIN CLOSE
EYNSHAM ROAD
FAIR VIEW
FERRY HINKSEY ROAD
FISHER ROW
FLORENCE PARK ROAD
FOX LANE
GIDLEY WAY
HAYNES ROAD
HEADLEY WAY
HEADLEY WAY
HELEN ROAD
HENLEY ROAD
HENLEY STREET
HENLEY STREET
HENRY ROAD
HERTFORD STREET
HIGH STREET
HIGH STREET
HOME CLOSE
HOPE WAY
HORWOOD CLOSE
HOWARD STREET
HOWARD STREET
HUGH ALLEN CRESCENT
HUTCHCOMB ROAD
JACKSON ROAD
KELLYS ROAD
KENILWORTH AVENUE
KINGSTON ROAD
LANHAM WAY
LAWN UPTON CLOSE
LEIDEN ROAD
LIME WALK
LONDON ROAD
LONDON ROAD
MAGDALEN ROAD
MAIN 

In [39]:
print(ox_venues.shape)
ox_venues.head()

(5129, 7)


,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ABINGDON ROAD,51.733564,-1.331408,The Bear & Ragged Staff,51.733540,-1.336767,Gastropub
1,ABINGDON ROAD,51.733564,-1.331408,The Vine Inn,51.733613,-1.331282,Pub
2,ABINGDON ROAD,51.733564,-1.331408,Link Security,51.730737,-1.331938,Home Service
3,ABINGDON ROAD,51.733564,-1.331408,Hid's Copse,51.746379,-1.314662,Forest
4,ABINGDON ROAD,51.733564,-1.331408,Denton Green,51.746820,-1.315124,Park


We have extracted 5129 venues. Now we will group them by street name

In [40]:
ox_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ABINGDON ROAD,7,7,7,7,7,7
ACREMEAD ROAD,7,7,7,7,7,7
ALDRICH ROAD,45,45,45,45,45,45
ARLINGTON DRIVE,50,50,50,50,50,50
ASHCROFT CLOSE,25,25,25,25,25,25
...,...,...,...,...,...,...
WOLSEY ROAD,46,46,46,46,46,46
WOOD AVENS WAY,43,43,43,43,43,43
WOODPERRY ROAD,4,4,4,4,4,4


### Let's see how many unique venues are in categories

In [44]:
print('Unique venue categories = ',format(len(ox_venues['Venue Category'].unique())))

Unique venue categories =  128


In [45]:
#one hot encoding
ox_onehot = pd.get_dummies(ox_venues[['Venue Category']], prefix = "", prefix_sep = "")

#add neighborhood column to the data frame
ox_onehot['Street'] = ox_venues['Street']

fixed_columns = [ox_onehot.columns[-1]]+list(ox_onehot.columns[:-1])
ox_onehot = ox_onehot[fixed_columns]
ox_onehot.head()

,Street,American Restaurant,Amphitheater,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bar,Beer Bar,Bookstore,Botanical Garden,Bowling Alley,Brazilian Restaurant,Brewery,Bridge,Burger Joint,Bus Station,Bus Stop,Café,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Library,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,...,Park,Parking,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Rest Area,Restaurant,Roof Deck,Rugby Pitch,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Stadium,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Warehouse Store,Wine Shop
0,ABINGDON ROAD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ABINGDON ROAD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ABINGDON ROAD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ABINGDON ROAD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ABINGDON ROAD,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
#new data frame size
ox_onehot.shape

(5129, 129)

In [47]:
#grouping rows by neighborhood and by taking mean frequency of occurence of each category
oxford_gr = ox_onehot.groupby('Street').mean().reset_index()
oxford_gr.head()

,Street,American Restaurant,Amphitheater,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bar,Beer Bar,Bookstore,Botanical Garden,Bowling Alley,Brazilian Restaurant,Brewery,Bridge,Burger Joint,Bus Station,Bus Stop,Café,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Library,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,...,Park,Parking,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Rest Area,Restaurant,Roof Deck,Rugby Pitch,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Stadium,Spanish Restaurant,Sporting Goods Shop,Sports Club,Sri Lankan Restaurant,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Warehouse Store,Wine Shop
0,ABINGDON ROAD,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.142857,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.142857,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
1,ACREMEAD ROAD,0.0,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.142857,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.142857,0.0,0.000000,0.142857,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
2,ALDRICH ROAD,0.0,0.0,0.00,0.00,0.00,0.0,0.044444,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.022222,0.022222,0.022222,0.0,0.0,0.0,0.00,0.0,0.066667,0.0,0.0,0.0,0.00,0.066667,0.00,0.0,0.0,0.022222,0.0,0.0,0.0,0.022222,0.0,...,0.044444,0.00,0.00,0.022222,0.022222,0.00,0.022222,0.0,0.066667,0.0,0.022222,0.000000,0.044444,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.022222,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00
3,ARLINGTON DRIVE,0.0,0.0,0.02,0.00,0.02,0.0,0.060000,0.0,0.0,0.04,0.0,0.0,0.0,0.00,0.02,0.0,0.000000,0.000000,0.040000,0.0,0.0,0.0,0.02,0.0,0.040000,0.0,0.0,0.0,0.04,0.060000,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.020000,0.00,0.00,0.000000,0.020000,0.02,0.000000,0.0,0.120000,0.0,0.000000,0.000000,0.040000,0.0,0.0,0.02,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.000000,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.02,0.00,0.00
4,ASHCROFT CLOSE,0.0,0.0,0.00,0.04,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.04,0.00,0.0,0.000000,0.040000,0.040000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.040000,0.04,0.0,0.0,0.000000,0.0,0.0,0.0,0.040000,0.0,...,0.040000,0.04,0.04,0.000000,0.000000,0.00,0.040000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.00,0.0,0.0,0.040000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.04,0.04


In [48]:
#print each neighborhood along the top 5 most common venues
num_top_venues = 5

for hood in oxford_gr['Street']:
    print("----"+hood+"----")
    temp = oxford_gr[oxford_gr['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ABINGDON ROAD----
           venue  freq
0         Forest  0.29
1  Historic Site  0.14
2           Park  0.14
3      Gastropub  0.14
4            Pub  0.14


----ACREMEAD ROAD----
             venue  freq
0              Pub  0.14
1              Gym  0.14
2             Food  0.14
3  Motorcycle Shop  0.14
4    Grocery Store  0.14


----ALDRICH ROAD----
                venue  freq
0               Hotel  0.16
1  Chinese Restaurant  0.07
2                 Pub  0.07
3       Grocery Store  0.07
4         Coffee Shop  0.07


----ARLINGTON DRIVE----
                venue  freq
0                 Pub  0.12
1              Bakery  0.06
2      History Museum  0.06
3         Coffee Shop  0.06
4  Chinese Restaurant  0.04


----ASHCROFT CLOSE----
                 venue  freq
0    Electronics Store  0.12
1        Grocery Store  0.08
2            Wine Shop  0.04
3            Pet Store  0.04
4  Sporting Goods Shop  0.04


----BANBURY ROAD----
                venue  freq
0               Hotel  0.16
1  

In [49]:
#create pandas data frame and display top 10 venues in each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Street'] = oxford_gr['Street']

for ind in np.arange(oxford_gr.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(oxford_gr.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABINGDON ROAD,Forest,Park,Pub,Historic Site,Home Service,Gastropub,Department Store,English Restaurant,Electronics Store,Eastern European Restaurant
1,ACREMEAD ROAD,Gym,Rest Area,Food,Motorcycle Shop,Café,Pub,Grocery Store,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,ALDRICH ROAD,Hotel,Grocery Store,Pub,Chinese Restaurant,Coffee Shop,Restaurant,Park,Gym,Gym / Fitness Center,Bakery
3,ARLINGTON DRIVE,Pub,Coffee Shop,History Museum,Bakery,Café,Chinese Restaurant,Hotel,Cocktail Bar,Thai Restaurant,Restaurant
4,ASHCROFT CLOSE,Electronics Store,Grocery Store,Forest,Parking,Park,IT Services,Warehouse Store,Golf Course,Furniture / Home Store,Wine Shop


In [50]:
neighborhoods_venues_sorted.shape

(123, 11)

## 4. Data modelling

After long process of data inspection and preparation we can move to the modelling stage. We will use k- Nearest Neighbors algorithm to cluster the neighborhoods by venues and facilities to determine the best location for the property search. We will create 5 clusters


In [73]:
#k-means for clustering
from sklearn.cluster import KMeans

kclusters = 5

ox_gr_clustering = oxford_gr.drop('Street', 1)

#run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ox_gr_clustering)

#check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted.head()

ValueError: ignored

In [74]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,ABINGDON ROAD,Forest,Park,Pub,Historic Site,Home Service,Gastropub,Department Store,English Restaurant,Electronics Store,Eastern European Restaurant
1,0,ACREMEAD ROAD,Gym,Rest Area,Food,Motorcycle Shop,Café,Pub,Grocery Store,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,0,ALDRICH ROAD,Hotel,Grocery Store,Pub,Chinese Restaurant,Coffee Shop,Restaurant,Park,Gym,Gym / Fitness Center,Bakery
3,2,ARLINGTON DRIVE,Pub,Coffee Shop,History Museum,Bakery,Café,Chinese Restaurant,Hotel,Cocktail Bar,Thai Restaurant,Restaurant
4,0,ASHCROFT CLOSE,Electronics Store,Grocery Store,Forest,Parking,Park,IT Services,Warehouse Store,Golf Course,Furniture / Home Store,Wine Shop


In [78]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ox_clustering= ox_av.join(neighborhoods_venues_sorted.set_index('Street'), on='Street')

ox_clustering

,Street,Postcode,Price_av,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,ABINGDON ROAD,OX2 9QN,455000.0,51.733564,-1.331408,1,Forest,Park,Pub,Historic Site,Home Service,Gastropub,Department Store,English Restaurant,Electronics Store,Eastern European Restaurant
5,ACREMEAD ROAD,OX33 1NZ,487500.0,51.745725,-1.154707,0,Gym,Rest Area,Food,Motorcycle Shop,Café,Pub,Grocery Store,Construction & Landscaping,Convenience Store,Cosmetics Shop
10,ALDRICH ROAD,OX2 7SS,492500.0,51.785151,-1.261481,0,Hotel,Grocery Store,Pub,Chinese Restaurant,Coffee Shop,Restaurant,Park,Gym,Gym / Fitness Center,Bakery
28,ARLINGTON DRIVE,OX3 0SL,497500.0,51.768291,-1.242917,2,Pub,Coffee Shop,History Museum,Bakery,Café,Chinese Restaurant,Hotel,Cocktail Bar,Thai Restaurant,Restaurant
33,ASHCROFT CLOSE,OX2 9SE,492500.0,51.750896,-1.312104,0,Electronics Store,Grocery Store,Forest,Parking,Park,IT Services,Warehouse Store,Golf Course,Furniture / Home Store,Wine Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,WOLSEY ROAD,OX2 7TA,468000.0,51.787217,-1.264100,0,Hotel,Chinese Restaurant,Coffee Shop,Grocery Store,Pub,Park,Bus Station,Gym,Gym / Fitness Center,Bakery
1135,WOOD AVENS WAY,OX3 9GA,545000.0,51.771410,-1.216918,0,Grocery Store,Pub,Café,Supermarket,Coffee Shop,Sandwich Place,Chinese Restaurant,Restaurant,Fast Food Restaurant,Sculpture Garden
1145,WOODPERRY ROAD,OX3 9UZ,550000.0,51.793918,-1.176281,2,Construction & Landscaping,Gastropub,Nature Preserve,Farm,Wine Shop,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega
1147,WOODSTOCK CLOSE,OX2 8DB,457500.0,51.784841,-1.273750,0,Hotel,Pub,Grocery Store,Coffee Shop,Chinese Restaurant,Gym,Gym / Fitness Center,Bakery,Park,Pharmacy


In [76]:
ox_clustering.shape

(139, 16)

## 5. Visualization and Discussion

To easier undestand our reasearch we can create the map of clusters and then examine each cluster on the most common venues.

In [80]:
#create the cluster map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ox_clustering['Latitude'], ox_clustering['Longitude'], 
                                  ox_clustering['Street'],ox_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill = True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [81]:
#Cluster 1
ox_clustering.loc[ox_clustering['Cluster Labels'] == 0, ox_clustering.columns[[1] + list(range(5, ox_clustering.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,OX33 1NZ,0,Gym,Rest Area,Food,Motorcycle Shop,Café,Pub,Grocery Store,Construction & Landscaping,Convenience Store,Cosmetics Shop
10,OX2 7SS,0,Hotel,Grocery Store,Pub,Chinese Restaurant,Coffee Shop,Restaurant,Park,Gym,Gym / Fitness Center,Bakery
33,OX2 9SE,0,Electronics Store,Grocery Store,Forest,Parking,Park,IT Services,Warehouse Store,Golf Course,Furniture / Home Store,Wine Shop
62,OX2 7JH,0,Hotel,Pub,Coffee Shop,Restaurant,Grocery Store,Chinese Restaurant,Indian Restaurant,Park,Gym / Fitness Center,Bakery
131,OX3 9FZ,0,Grocery Store,Café,Pub,Supermarket,Coffee Shop,Sandwich Place,Chinese Restaurant,Restaurant,Fast Food Restaurant,Sculpture Garden
...,...,...,...,...,...,...,...,...,...,...,...,...
1117,OX3 0PL,0,Grocery Store,Chinese Restaurant,Hotel,Restaurant,Pub,Bakery,Sandwich Place,Pool,Indian Restaurant,Café
1119,OX4 2JL,0,Grocery Store,Pub,Supermarket,Coffee Shop,Gym / Fitness Center,Furniture / Home Store,Pizza Place,Clothing Store,Shopping Mall,Go Kart Track
1133,OX2 7TA,0,Hotel,Chinese Restaurant,Coffee Shop,Grocery Store,Pub,Park,Bus Station,Gym,Gym / Fitness Center,Bakery
1135,OX3 9GA,0,Grocery Store,Pub,Café,Supermarket,Coffee Shop,Sandwich Place,Chinese Restaurant,Restaurant,Fast Food Restaurant,Sculpture Garden


In [82]:
#Cluster 2
ox_clustering.loc[ox_clustering['Cluster Labels'] == 1, ox_clustering.columns[[1] + list(range(5, ox_clustering.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,OX2 9QN,1,Forest,Park,Pub,Historic Site,Home Service,Gastropub,Department Store,English Restaurant,Electronics Store,Eastern European Restaurant
107,OX2 9PS,1,Pub,Campground,Forest,Gastropub,Bus Stop,Wine Shop,Eastern European Restaurant,Farm,English Restaurant,Electronics Store
170,OX4 1QN,1,Pub,Café,Turkish Restaurant,Asian Restaurant,Ice Cream Shop,Burger Joint,Wine Shop,Plaza,Roof Deck,Eastern European Restaurant
243,OX4 3NN,1,Pub,Park,Turkish Restaurant,Burger Joint,Caribbean Restaurant,Hookah Bar,Record Shop,Portuguese Restaurant,Canal Lock,Hotel
257,OX4 2DL,1,Pub,Café,Turkish Restaurant,Asian Restaurant,Park,Wine Shop,Canal Lock,Eastern European Restaurant,Hotel,Record Shop
309,OX4 1LN,1,Pub,Burger Joint,Café,Turkish Restaurant,Asian Restaurant,Ice Cream Shop,Dessert Shop,Eastern European Restaurant,Record Shop,Portuguese Restaurant
329,OX4 1XR,1,Pub,Café,Turkish Restaurant,Burger Joint,Asian Restaurant,Ice Cream Shop,Coffee Shop,Record Shop,Portuguese Restaurant,Plaza
388,OX4 3PW,1,Pub,Burger Joint,Pizza Place,Park,Wine Shop,Noodle House,Record Shop,Portuguese Restaurant,Caribbean Restaurant,Chinese Restaurant
463,OX3 7SU,1,Pub,Café,Park,Sandwich Place,Burger Joint,Cocktail Bar,Supermarket,Coffee Shop,Asian Restaurant,Turkish Restaurant
464,OX3 7TA,1,Pub,Café,Park,Sandwich Place,Burger Joint,Cocktail Bar,Supermarket,Coffee Shop,Asian Restaurant,Turkish Restaurant


In [83]:
#Cluster 3
ox_clustering.loc[ox_clustering['Cluster Labels'] == 2, ox_clustering.columns[[1] + list(range(5, ox_clustering.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,OX3 0SL,2,Pub,Coffee Shop,History Museum,Bakery,Café,Chinese Restaurant,Hotel,Cocktail Bar,Thai Restaurant,Restaurant
116,OX2 7HJ,2,Pub,Hotel,Chinese Restaurant,Coffee Shop,Restaurant,Indian Restaurant,Bakery,Deli / Bodega,Church,Pharmacy
139,OX2 0EZ,2,Pub,Bakery,Restaurant,Coffee Shop,Cocktail Bar,Indian Restaurant,History Museum,Thai Restaurant,Ice Cream Shop,Chinese Restaurant
140,OX2 0HW,2,Pub,Bakery,Restaurant,Coffee Shop,Cocktail Bar,Indian Restaurant,History Museum,Thai Restaurant,Ice Cream Shop,Chinese Restaurant
246,OX3 0SG,2,Pub,Restaurant,Hotel,Grocery Store,Chinese Restaurant,Bakery,Indian Restaurant,Sandwich Place,History Museum,French Restaurant
286,OX2 7BL,2,Pub,Bakery,Coffee Shop,History Museum,Hotel,Restaurant,Chinese Restaurant,Cocktail Bar,Park,Middle Eastern Restaurant
324,OX2 0HX,2,Pub,Bakery,History Museum,Restaurant,Coffee Shop,Thai Restaurant,Cocktail Bar,Ice Cream Shop,Bookstore,Hotel
336,OX3 0HE,2,Pub,Restaurant,History Museum,Bakery,Park,Ice Cream Shop,Bookstore,Coffee Shop,Roof Deck,Record Shop
368,OX2 0BY,2,Pub,Bakery,Coffee Shop,Cocktail Bar,Café,Thai Restaurant,Bookstore,History Museum,Ice Cream Shop,Restaurant
380,OX1 1HT,2,Pub,Café,History Museum,Bakery,Sandwich Place,Cocktail Bar,Coffee Shop,Restaurant,Bookstore,Ice Cream Shop


In [84]:
#Cluster 4
ox_clustering.loc[ox_clustering['Cluster Labels'] == 3, ox_clustering.columns[[1] + list(range(5, ox_clustering.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
251,OX4 4JB,3,Pub,Grocery Store,Pizza Place,Hotel,Turkish Restaurant,Clothing Store,Furniture / Home Store,Coffee Shop,Outdoor Supply Store,Cocktail Bar
261,OX33 1HW,3,Pub,Rest Area,Farm,Wine Shop,Fast Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
285,OX1 5JP,3,Pub,Furniture / Home Store,Gastropub,Department Store,Farm,English Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Deli / Bodega
338,OX4 4JF,3,Pub,Grocery Store,Pizza Place,Clothing Store,Furniture / Home Store,Wine Shop,Noodle House,Cocktail Bar,College Gym,Park
395,OX1 5DS,3,Pub,Campground,Wine Shop,Fast Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
411,OX33 1TD,3,Grocery Store,Pub,Home Service,Motorcycle Shop,Wine Shop,Dessert Shop,Farm,English Restaurant,Electronics Store,Eastern European Restaurant
667,OX1 5QY,3,Pub,Restaurant,Grocery Store,Hotel,Deli / Bodega,English Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Department Store
838,OX1 5QS,3,Pub,Restaurant,Grocery Store,Hotel,Deli / Bodega,English Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Department Store
912,OX4 4JL,3,Pub,Grocery Store,Hotel,Pizza Place,Furniture / Home Store,Wine Shop,Outdoor Supply Store,Clothing Store,Cocktail Bar,College Gym
1047,OX4 4EX,3,Pub,Grocery Store,Hotel,Pizza Place,Coffee Shop,Furniture / Home Store,Outdoor Supply Store,Clothing Store,Cocktail Bar,College Gym


In [86]:
#Cluster 5
ox_clustering.loc[ox_clustering['Cluster Labels'] == 4, ox_clustering.columns[[1] + list(range(5, ox_clustering.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
732,OX44 7UU,4,Hotel,Gas Station,Food & Drink Shop,Wine Shop,Farm,English Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Department Store
1088,OX44 7UT,4,Hotel,Gas Station,Food & Drink Shop,Wine Shop,Farm,English Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Department Store


### Discussion

Let's look at the clusters venues. 
Cluster 1 has plenty of grocery stores and supermarkets, parks, hotels, restaraunts bus station and the shopping mall.
Cluster 2 has a lot of restaraunts/cafes, museums, historic site, bus stops and karting.
Cluster 3 has museums, restaurants, the Canal, coffee shops and book store.
Cluster 4 has a grocery stores, some hotels, gym, a few number of restaraunts and no parks or green area.
Cluster 5 is a remote location with a limited number of venues.

From the venues/amenities point of view cluster 1 and 2 are the most attractive as they contain not only restaraunts, grocery stores, hotels, but also a transportation facilities.


## 6. Conclusion

As the housing market slowing down due to pandemic and prices are droping it is a good time to make an investment as per expert's opinion the market will come back after the pandemic. According to McKenzie research up to 60% of property performance is coming from non-traditional features such as facilities and amenities of the location.
With the lock down in place it is not possible to study the area on the foot, it is also would take a longer time to do that in a normal conditions.

We have used data science methdology to cluster the area of Oxford city to define the most attractive locations for the investment.
The data used for the research were downloaded from public HM Land Registry data base. FourSquare API was used to define most common amenities/venues. 

We followed the workflow of exploring the data, understanding it, pre-processing and processing to create Pandas data frames. We then used k-Nearest Neighbours algorithm to find the similarities of the properties locations and created 5 clusters.

We have visualized the area in terms of average property price distribution and cluster spread by creating the maps.

From the cluster examination we found that cluster 1 and 2 are most attractive for the investor as they have most diverse venues, though some may look for different features when investing on property market.